In [ ]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
from numpy.fft import fft, ifft
import numpy as np
import scipy
import seaborn as sns
import re
from tqdm.notebook import tqdm

#IMPORTING FUNCTIONS FOR CLASSIFIERS
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedShuffleSplit, TimeSeriesSplit, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

#IMPORTING FUNCTIONS FOR CLASSIFIERS METRICS
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

#OPTMIZE PARAMETHERS
import optuna

#IMPORTING FUNCTIONS FROM ./defs

from defs import defs_

# Importing Database

In [2]:
samples = 1024;

df_final = pd.read_csv('df_csv/' + str(samples) + '_samples/dataframe_both_final', index_col=0, dtype='unicode');
df_quick_test = pd.read_csv('df_csv/' + str(samples) + '_samples/dataframe_both_simples', index_col=0, dtype='unicode');
df_finalB = pd.read_csv('df_csv/' + str(samples) + '_samples/dataframeB_final', index_col=0, dtype='unicode');

df_final

,s_max_S1,s_sum_S1,s_mean_S1,s_std_S1,psd_max_S1,psd_sum_S1,psd_mean_S1,f*s_mean_S1,f*s_max_S1,f*s_sum_S1,...,f*s_sum_S30,high_freq_S30,low_freq_S30,mean_S30,std_S30,median_S30,skew_S30,amp_max_min_S30,kurtosis_S30,Scenario
0,0.0007781732421875,0.10603555506398436,0.00020750597859879523,0.00016167695106839307,3.027767974283028e-07,1.7680131837389394e-05,3.459908383050762e-08,0.053121666512757876,0.39764754473552705,13871.182681477849,...,13385.514565316656,365.0009344023921,0.00273971901369863,-0.0009625774402050785,0.14051081616489033,-0.00160728,-0.028261960420297376,0.9763257999999999,0.3741716150735144,1
1,0.0008467341796875,0.13023888669609376,0.000254870619757522,0.00018445991267254926,3.584793855255318e-07,2.529053775678008e-05,4.9492246099373935e-08,0.06521766114573235,0.43248741145997843,17029.700896034778,...,13010.161882121558,399.82088024564996,0.00250112,-0.0007408178286132816,0.13220517439369525,0.0031077394999999997,-0.13960095714872756,0.8766658,-0.19597598823181217,1
2,0.001176632421875,0.12327197006771484,0.0002412367320307531,0.0001890431227999188,6.922319281037139e-07,2.3999744337079164e-05,4.696623157941128e-08,0.0617289488307966,0.6009899240921317,16118.724847646437,...,13829.263648311005,61.97223643807574,0.01613625806451613,-0.0010596247472656256,0.14178383623011304,-0.003839492,0.15268936763001495,0.8964205000000001,0.019830716117583247,1
3,0.0012122330078125,0.1757829419080078,0.0003439979293698783,0.00024472774799294946,7.347544326150705e-07,4.553680586538265e-05,8.911312302423219e-08,0.08802403515094125,0.6191736771848088,22984.92408264893,...,18246.993570585375,447.79938587512794,0.002233142857142857,-0.0006096456118164065,0.16496941710744412,-0.0008257963500000001,-0.03433807485017546,1.0781062000000001,-0.09253528403808797,1
4,0.00139026171875,0.16444245279882813,0.0003218051913871392,0.00023980974147593974,9.664138233108518e-07,4.115269344082516e-05,8.053364665523515e-08,0.08234523832833655,0.7101056109675364,21502.07097753357,...,15276.911632174782,114.94850307062435,0.008699547826086958,-0.0008355930417968758,0.1391090713471743,0.005240619,-0.10385369203581078,1.0376701,0.058457058525502426,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7931,0.0010174564453125,0.15082474249876954,0.0002951560518566919,0.00022261285064018418,5.17608809053974e-07,3.492009872364257e-05,6.833678810888956e-08,0.07378901296417295,0.5077346128463744,19267.860854217804,...,15653.785313898728,405.2734375,0.0024674698795180724,0.0022268341846679663,0.1463088548130687,0.004260801,-0.06072306121432907,0.9396711,-0.07285194834540976,0
7932,0.000901896875,0.13406956425917968,0.00026236705334477435,0.00019337784726412683,4.067089865673828e-07,2.714213866704521e-05,5.311573124666382e-08,0.06559176333619357,0.4500676788330078,17127.386834110202,...,14410.321273549074,202.1484375,0.0049468599033816426,0.0016730884660156261,0.1432019711880003,0.009320239000000001,-0.0628559234161243,1.0328667,0.23811567861600835,0
7933,0.0013240921875,0.13888234787558593,0.0002717854165862738,0.00020265458940192055,8.766100604992674e-07,2.9366197892669353e-05,5.746809763731772e-08,0.06794635414656847,0.6607530349731445,17742.219941106105,...,16386.130810058105,250.0,0.004,0.001823433908984375,0.15540852807998115,0.005102217500000001,0.12789408137547786,1.0024451,-0.11665106960237459,0
7934,0.001221470703125,0.14190300879804688,0.00027769669040713675,0.0002164993447214097,7.459953392963409e-07,3.167878533167256e-05,6.199370906393848e-08,0.06942417260178421,0.6095425090789794,18128.109373950494,...,14601.659228655762,295.8984375,0.0033795379537953796,0.001275242775927735,0.1362552667126023,-5.1783374999999994e-05,0.02412409924378543,0.8374804,-0.05377166902358077,0


# Criando dataframe voltada para 1 sensor

In [4]:
df_ready = defs_.createScaledDataframeScenario(df_final, 1);

variation = 'S1';
columns_drop = [
    's_sum_'+variation,
    's_mean_'+variation,
    's_std_'+variation,
    'psd_max_'+variation,
    'psd_sum_'+variation,
    'f*s_mean_'+variation,
    'f*s_max_'+variation,
    'f*s_sum_'+variation,
    'std_'+variation
    ]
df_ready = df_ready.drop(columns=columns_drop);

# Modelo sem Cross validation
### Preparação dos inputs e target
### Setting X and y for model score:0.5545965387932191
# print('Scenario is: ' + str(1))
# Setting target variable
y_new_attemp = df_ready['damaged'];
y_new_attemp = y_new_attemp.astype(int);
# Setting other variables
X_new_attemp = df_ready.drop(columns=['damaged']);
X_new_attemp = X_new_attemp.astype(float);
# Separate database for test and train and 
X_train, X_test, y_train, y_test = train_test_split(X_new_attemp, y_new_attemp, test_size=0.3, shuffle=True);
#Scale df
std_scaler = StandardScaler()
X_scaled_train = pd.DataFrame(std_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index);
X_scaled_test = pd.DataFrame(std_scaler.transform(X_test),columns=X_train.columns,index=X_test.index);
# print('X_scaled_train, y_train e X_scaled_test, y_test estão prontos.')

### Treinando modelo sem otimização
### DEFAULT PARAMETHERS ON FULL DATAFRAME
rf = RandomForestClassifier(n_estimators = 5000);
rf.fit(X_scaled_train,y_train);
### Predict df test
y_pred = rf.predict(X_scaled_test);
accuracy_test = accuracy_score(y_test, y_pred);
print('Preformance de previsão de df de teste Rf(n_esti=5000): ' + "{:.4f}".format(accuracy_test))
### Predict df train (overfit)
y_pred = rf.predict(X_scaled_train);
accuracy_train = accuracy_score(y_train, y_pred);
print('Performance de previsão de df de treino Rf(n_esti=5000): ' + "{:.4f}".format(accuracy_train))
if (accuracy_train > accuracy_test) :
    print('-> overfit(?)')

### Treinando modelo sem otimização
### DEFAULT PARAMETHERS ON FULL DATAFRAME
rf = RandomForestClassifier(n_estimators = 10000);
rf.fit(X_scaled_train,y_train);
### Predict df test
y_pred = rf.predict(X_scaled_test);
accuracy_test = accuracy_score(y_test, y_pred);
print('Preformance de previsão de df de teste Rf(n_esti=10000): ' + "{:.4f}".format(accuracy_test))
### Predict df train (overfit)
y_pred = rf.predict(X_scaled_train);
accuracy_train = accuracy_score(y_train, y_pred);
print('Performance de previsão de df de treino Rf(n_esti=10000): ' + "{:.4f}".format(accuracy_train))
if (accuracy_train > accuracy_test) :
    print('-> overfit(?)')

# Modelo com Cross validation
### Preparação dos inputs e target
### Setting X and y for model
# print('Scenario is: ' + str(1))
# Setting target variable
y_new_attemp = df_ready['damaged'];
y_new_attemp = y_new_attemp.astype(int);
# Setting other variables
X_new_attemp = df_ready.drop(columns=['damaged']);
X_new_attemp = X_new_attemp.astype(float);
#Scale df
std_scaler = StandardScaler()
X_scaled = pd.DataFrame(std_scaler.fit_transform(X_new_attemp), columns=X_new_attemp.columns, index=X_new_attemp.index);
# print('X_scaled_train, y_train e X_scaled_test, y_test estão prontos.')
### DEFAULT PARAMETHERS ON FULL DATAFRAME

score_new_attemp = cross_val_score(RandomForestClassifier(n_estimators = 5000), X_scaled, y_new_attemp);
print('Performance da media das validações cruzadas Rf(n_esti=5000): ' + "{:.4f}".format(np.mean(score_new_attemp)))

score_new_attemp = cross_val_score(RandomForestClassifier(n_estimators = 10000), X_scaled, y_new_attemp);
print('Performance da media das validações cruzadas Rf(n_esti=10000): ' + "{:.4f}".format(np.mean(score_new_attemp)))

Preformance de previsão de df de teste Rf(n_esti=5000): 0.6299
Performance de previsão de df de treino Rf(n_esti=5000): 1.0000
-> overfit(?)
Preformance de previsão de df de teste Rf(n_esti=10000): 0.6364
Performance de previsão de df de treino Rf(n_esti=10000): 1.0000
-> overfit(?)
Performance da media das validações cruzadas Rf(n_esti=5000): 0.6584
Performance da media das validações cruzadas Rf(n_esti=10000): 0.6525


In [7]:
### OPTMAZING PARAMETHERS TO MAXIMIZE SCORE ON QUICK DATAFRAME {'n_estimators': 800, 'max_depth': 2}
### last score: 0.5761515497421756

def objective(trial):
    ### PARAMS
    # param_n_estimators = trial.suggest_int('n_estimators',100,100000,100);
    param_max_features = trial.suggest_int('max_features',1,qty_coef,1);
    param_max_depth = trial.suggest_int('max_depth',1,50,1);
    # param_min_samples_split = trial.suggest_int('min_samples_split',1,20,1);

    rf_com_cross_optuna = RandomForestClassifier(n_estimators = n_est, bootstrap=True, max_samples=0.8, max_features=param_max_features, max_depth=param_max_depth);
    score_new_attemp = cross_val_score(rf_com_cross_optuna, X_new_attemp, y_new_attemp);

    return np.mean(score_new_attemp)

## Create coef models

In [1]:
#PATHS USED
path_quick_test = [
    "./database_raw/DatasetA/zzzAD1.txt",
    "./database_raw/DatasetA/zzzAD30.txt",
    "./database_raw/DatasetA/zzzAD12.txt",
    "./database_raw/DatasetA/zzzAU.txt",
];
path = glob.glob("./database_raw/DatasetA/zzzA*.txt", recursive=True);
pathB = glob.glob("./database_raw/DatasetB/zzzB*.txt", recursive=True);

# Qty of row to be aggrouped 
samples = 1024;

qty_coef = 30;

df_final = pd.read_csv('df_csv/' + str(samples) + '_samples/df_reg_coef/dataframe_regression_coef_'+str(qty_coef)+'_final', index_col=0, dtype='unicode');
# df_quick_test = pd.read_csv('df_csv/' + str(samples) + '_samples/df_reg_coef/dataframe_regression_coef_'+str(qty_coef)+'_simples', index_col=0, dtype='unicode');
dfB_final = pd.read_csv('df_csv/' + str(samples) + '_samples/df_reg_coef/dataframeB_regression_coef_'+str(qty_coef)+'_final', index_col=0, dtype='unicode');

df_final

NameError: name 'glob' is not defined

In [8]:
n_est = 500;
optuna_n_trials = 50;
head_columns = [
    'Sensor',
    'acc_cross',
    'acc_sem',
    'pod',
    'param',
    'acc_com_cross_optuna',
    'acc_sem_cross_optuna',
    'pod_optuna',
];
grouped_values = pd.DataFrame(columns=head_columns);

for path_new in tqdm(path):
    scenario = defs_.getDamageScenarioLabel(path_new);
    df_ready = defs_.createScaledDataframeScenario(df_final, scenario);
    new_row = [];
    new_row.append('S' + str(scenario));
    print('Sensor: S' + str(scenario) + ' and Scenario: ' + str(scenario));
    
    if scenario == 0:
        break
    
    ### Setting X and y for model score
    # Setting target variable
    y_new_attemp = df_ready['damaged'];
    y_new_attemp = y_new_attemp.astype(int);
    # Setting other variables
    X_new_attemp = df_ready.drop(columns=['damaged']);
    X_new_attemp = X_new_attemp.astype(float);

    ### Modelo com Cross validation 
    rf_com_cross = RandomForestClassifier(n_estimators = n_est);
    score_new_attemp = cross_val_score(rf_com_cross, X_new_attemp, y_new_attemp);
    new_row.append("{:.4f}".format(np.mean(score_new_attemp)));
    print('Performance da media das validações cruzadas: ' + "{:.4f}".format(np.mean(score_new_attemp)))

    # Modelo sem Cross validation 
    X_train, X_test, y_train, y_test = train_test_split(X_new_attemp, y_new_attemp, test_size=0.3, shuffle=True);
    X_train_sem_cross = X_train;
    X_test_sem_cross = X_test;
    y_train_sem_cross = y_train;
    y_test_sem_cross = y_test;
    ### DEFAULT PARAMETHERS ON FULL DATAFRAME
    rf_sem_cross = RandomForestClassifier(n_estimators = n_est);
    rf_sem_cross.fit(X_train_sem_cross,y_train_sem_cross);
    ### Predict df test
    y_pred = rf_sem_cross.predict(X_test_sem_cross);
    accuracy_test = accuracy_score(y_test_sem_cross, y_pred);
    print('Preformance de previsão de df de teste: ' + "{:.4f}".format(accuracy_test));
    new_row.append("{:.4f}".format(accuracy_test));
    ## Predict df train (overfit)
    y_pred_overfit = rf_sem_cross.predict(X_train_sem_cross);
    accuracy_train = accuracy_score(y_train_sem_cross, y_pred_overfit);
    print('-> overfit(?): ' + "{:.4f}".format(accuracy_train));

    #PATHB
    for path_b in pathB:
        index = defs_.getDamageScenarioLabel(path_b);
        if ( index == scenario):
            path_used = path_b;
            break
       
    df_scenario = defs_.createDatabase(path_used);
    df_scenario = df_scenario.astype(float);
    # [index,    'Time',      'S1'  , ..., 'Scenario']
    # [  0,     0.000000,   0.004954, ...,     1.0   ]
    # [ ...,       ...,        ...  , ...,     ...   ]
    # [ 262k,   255.9999,  -0.211089, ...,     1.0   ]
    sensor = 3;
    sensor_label = 'S'+ str(scenario);
    df_scenario_S1 = defs_.getRegressionCaracteristics(df_scenario,sensor_label,qty_coef=qty_coef);
    df_scenario_S1 = df_scenario_S1.astype(float);

    ### Predict df test
    y_pred = rf_sem_cross.predict(df_scenario_S1);
    new_row.append("{:.4f}".format(defs_.probabilityOfDamage(y_pred)));
    print('Probability of damage: ' + "{:.4f}".format(defs_.probabilityOfDamage(y_pred)));
    
    study = optuna.create_study(direction='maximize');
    study.optimize(objective, n_trials=optuna_n_trials);
    new_row.append(study.best_params);
    print(study.best_params);

    ### Modelo com Cross validation e 
    rf_com_cross = RandomForestClassifier(n_estimators = n_est, bootstrap=True, max_samples=0.8, max_features= new_row[4]['max_features'], max_depth= new_row[4]['max_depth']);
    score_new_attemp = cross_val_score(rf_com_cross, X_new_attemp, y_new_attemp);
    new_row.append("{:.4f}".format(np.mean(score_new_attemp)));
    print('Performance da media das validações cruzadas: ' + "{:.4f}".format(np.mean(score_new_attemp)))

    ### Modelo com optuna sem Cross validation
    rf_sem_cross = RandomForestClassifier(n_estimators = n_est, bootstrap=True, max_samples=0.8,max_features= new_row[4]['max_features'], max_depth= new_row[4]['max_depth']);
    rf_sem_cross.fit(X_train_sem_cross,y_train_sem_cross);
    ### Predict df test
    y_pred = rf_sem_cross.predict(X_test_sem_cross);
    accuracy_test = accuracy_score(y_test_sem_cross, y_pred);
    print('Preformance de previsão de df de teste: ' + "{:.4f}".format(accuracy_test));
    new_row.append("{:.4f}".format(accuracy_test));
    ## Predict df train (overfit)
    y_pred_overfit = rf_sem_cross.predict(X_train_sem_cross);
    accuracy_train = accuracy_score(y_train_sem_cross, y_pred_overfit);
    print('-> overfit(?): ' + "{:.4f}".format(accuracy_train));

    ### Predict df test
    y_pred = rf_sem_cross.predict(df_scenario_S1);
    new_row.append("{:.4f}".format(defs_.probabilityOfDamage(y_pred)));
    print('Probability of damage: ' + "{:.4f}".format(defs_.probabilityOfDamage(y_pred)));
    
    array_features = pd.DataFrame([new_row], columns=grouped_values.columns);
    grouped_values = pd.concat([grouped_values, array_features], ignore_index=True);

# time: 1h 16m 6s
grouped_values

  0%|          | 0/31 [00:00<?, ?it/s]

Sensor: S1 and Scenario: 1
Performance da media das validações cruzadas: 0.9941
Preformance de previsão de df de teste: 0.9805
-> overfit(?): 1.0000


[I 2023-06-19 23:49:45,168] A new study created in memory with name: no-name-0b9201a5-8a30-4fb3-88dd-1c2d5d840aaa


Probability of damage: 1.0000


[I 2023-06-19 23:49:48,532] Trial 0 finished with value: 0.9901960784313726 and parameters: {'max_features': 20, 'max_depth': 43}. Best is trial 0 with value: 0.9901960784313726.
[I 2023-06-19 23:49:50,526] Trial 1 finished with value: 0.9901960784313726 and parameters: {'max_features': 1, 'max_depth': 44}. Best is trial 0 with value: 0.9901960784313726.
[I 2023-06-19 23:49:54,333] Trial 2 finished with value: 0.9901960784313726 and parameters: {'max_features': 27, 'max_depth': 40}. Best is trial 0 with value: 0.9901960784313726.
[I 2023-06-19 23:49:57,040] Trial 3 finished with value: 0.9921568627450981 and parameters: {'max_features': 8, 'max_depth': 12}. Best is trial 3 with value: 0.9921568627450981.
[I 2023-06-19 23:49:59,802] Trial 4 finished with value: 0.9941176470588236 and parameters: {'max_features': 11, 'max_depth': 29}. Best is trial 4 with value: 0.9941176470588236.
[I 2023-06-19 23:50:03,538] Trial 5 finished with value: 0.9901960784313726 and parameters: {'max_features'

...

[I 2023-06-20 01:05:43,331] A new study created in memory with name: no-name-118c4159-9aaa-4645-8ed2-0777d3157d16


Probability of damage: 0.9961


[I 2023-06-20 01:05:47,784] Trial 0 finished with value: 0.9764896249762041 and parameters: {'max_features': 27, 'max_depth': 43}. Best is trial 0 with value: 0.9764896249762041.
[I 2023-06-20 01:05:50,424] Trial 1 finished with value: 0.9901960784313726 and parameters: {'max_features': 9, 'max_depth': 3}. Best is trial 1 with value: 0.9901960784313726.
[I 2023-06-20 01:05:53,082] Trial 2 finished with value: 0.9941176470588236 and parameters: {'max_features': 5, 'max_depth': 32}. Best is trial 2 with value: 0.9941176470588236.
[I 2023-06-20 01:05:55,457] Trial 3 finished with value: 1.0 and parameters: {'max_features': 1, 'max_depth': 20}. Best is trial 3 with value: 1.0.
[I 2023-06-20 01:05:57,904] Trial 4 finished with value: 0.996078431372549 and parameters: {'max_features': 4, 'max_depth': 27}. Best is trial 3 with value: 1.0.
[I 2023-06-20 01:06:02,211] Trial 5 finished with value: 0.9745288406624786 and parameters: {'max_features': 27, 'max_depth': 36}. Best is trial 3 with valu

{'max_features': 1, 'max_depth': 20}
Performance da media das validações cruzadas: 1.0000
Preformance de previsão de df de teste: 0.9935
-> overfit(?): 1.0000
Probability of damage: 0.9883
Sensor: S0 and Scenario: 0


,Sensor,acc_cross,acc_sem,pod,param,acc_com_cross_optuna,acc_sem_cross_optuna,pod_optuna
0,S1,0.9941,0.9805,1.0000,"{'max_features': 11, 'max_depth': 29}",0.9922,0.9805,1.0000
1,S10,0.9980,1.0000,1.0000,"{'max_features': 7, 'max_depth': 25}",0.9980,1.0000,1.0000
2,S11,0.9941,0.9935,1.0000,"{'max_features': 14, 'max_depth': 42}",0.9922,0.9935,1.0000
3,S12,0.9922,1.0000,1.0000,"{'max_features': 1, 'max_depth': 44}",0.9902,1.0000,1.0000
4,S13,0.9648,0.9610,1.0000,"{'max_features': 1, 'max_depth': 25}",0.9687,0.9610,0.9961
5,S14,0.9922,1.0000,1.0000,"{'max_features': 8, 'max_depth': 39}",0.9902,1.0000,1.0000
6,S15,0.9804,0.9870,0.9961,"{'max_features': 7, 'max_depth': 50}",0.9824,0.9870,0.9922
7,S16,1.0000,1.0000,1.0000,"{'max_features': 3, 'max_depth': 17}",1.0000,1.0000,1.0000
8,S17,0.9980,1.0000,1.0000,"{'max_features': 9, 'max_depth': 5}",0.9980,1.0000,0.9883
9,S18,1.0000,1.0000,1.0000,"{'max_features': 15, 'max_depth': 27}",1.0000,0.9935,0.9922


In [9]:
grouped_values.to_csv('results_final_coef_'+str(qty_coef)+'.csv')